In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from tqdm import tqdm


In [8]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        
        self.fc_layers = nn.Sequential(
            nn.Linear(128 * 28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 1),
        )
        
    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)  # Flatten the output from conv layers
        x = self.fc_layers(x)
        return x
    

In [9]:
# Data transformations (you might need to adjust these based on your dataset)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# Load data
train_data = datasets.ImageFolder(root='data/train/', transform=transform)
valid_data = datasets.ImageFolder(root='data/valid/', transform=transform)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=32, shuffle=False)

# Initialize model, loss function, and optimizer
model = CNNModel()
loss_fn = nn.BCEWithLogitsLoss()  # Binary cross-entropy loss with logits
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [10]:
# Training settings
EPOCHS = 10
train_losses = []
valid_losses = []
best_valid_loss = float('inf')  # Initialize with infinity to ensure any loss is smaller
best_model_wts = None  # To store best model weights

print("Starting training...")

for epoch in range(EPOCHS):
    model.train()
    train_loss = 0.0
    correct_train = 0  # To calculate training accuracy
    
    for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{EPOCHS}", unit="batch"):
        optimizer.zero_grad()
        outputs = model(inputs).squeeze()
        loss = loss_fn(outputs, labels.float())
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        
        # Calculate training accuracy
        preds = torch.round(outputs) 
        correct_train += (preds == labels.float()).sum().item()
    
    train_losses.append(train_loss/len(train_loader))
    train_acc = correct_train / len(train_loader.dataset)
    
    model.eval()
    valid_loss = 0.0
    correct_valid = 0  # To calculate validation accuracy
    
    with torch.no_grad():
        for inputs, labels in valid_loader:
            outputs = model(inputs).squeeze()
            loss = loss_fn(outputs, labels.float())
            valid_loss += loss.item()
            
            # Calculate validation accuracy
            preds = torch.round(outputs)
            correct_valid += (preds == labels.float()).sum().item()
    
    valid_losses.append(valid_loss/len(valid_loader))
    valid_acc = correct_valid / len(valid_loader.dataset)
    
    # Save the model if it has a lower validation loss than the best model seen so far
    if valid_losses[-1] < best_valid_loss:
        best_valid_loss = valid_losses[-1]
        best_model_wts = model.state_dict().copy()
    
    print(f"Epoch {epoch}/{EPOCHS} - Train Loss: {train_losses[-1]:.4f}, Train Acc: {train_acc:.4f}, Valid Loss: {valid_losses[-1]:.4f}, Valid Acc: {valid_acc:.4f}")

# Load the best model weights
model.load_state_dict(best_model_wts)
print("Training completed.")



Starting training...


Epoch 0/10:   0%|          | 0/7 [00:00<?, ?batch/s]

Epoch 0/10: 100%|██████████| 7/7 [00:12<00:00,  1.74s/batch]


Epoch 0/10 - Train Loss: 0.8568, Train Acc: 0.2903, Valid Loss: 0.6847, Valid Acc: 0.5000


Epoch 1/10: 100%|██████████| 7/7 [00:14<00:00,  2.10s/batch]


...

Epoch 7/10 - Train Loss: 0.1270, Train Acc: 0.0461, Valid Loss: 0.6341, Valid Acc: 0.0179


Epoch 8/10: 100%|██████████| 7/7 [00:15<00:00,  2.25s/batch]


Epoch 8/10 - Train Loss: 0.0752, Train Acc: 0.0046, Valid Loss: 0.5196, Valid Acc: 0.0179


Epoch 9/10: 100%|██████████| 7/7 [00:15<00:00,  2.24s/batch]


Epoch 9/10 - Train Loss: 0.0747, Train Acc: 0.0138, Valid Loss: 0.7864, Valid Acc: 0.0536
Training completed.
